In [1]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.apg import train as apg
from brax.training.agents.apg import networks as apg_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [2]:
from utils.SimpleConverter import SimpleConverter
from agent_mimic_env.agent_template import HumanoidTemplate
from agent_mimic_env.agent_eval_template import HumanoidEvalTemplate
from agent_mimic_env.agent_test_apg import HumanoidAPGTest
from utils.util_data import *
from copy import deepcopy

In [3]:
import agent_mimic_env
from agent_mimic_env import register_mimic_env


In [4]:

import distutils.util
import os
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


In [5]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.8" # 0.9 causes too much lag. 
from datetime import datetime
import functools

# Math
import jax.numpy as jp
import numpy as np
import jax
from jax import config # Analytical gradients work much better with double precision.
config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)
config.update('jax_default_matmul_precision', jax.lax.Precision.HIGH)
from brax import math

# Sim
import mujoco
import mujoco.mjx as mjx

# Brax
from brax import envs
from brax.base import Motion, Transform
from brax.io import mjcf
from brax.envs.base import PipelineEnv, State
from brax.mjx.pipeline import _reformat_contact
from brax.training.acme import running_statistics
from brax.io import model

# Algorithms
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.training.agents.ppo import train as ppo

# Supporting
from etils import epath
import mediapy as media
import matplotlib.pyplot as plt
from ml_collections import config_dict
from typing import Any, Dict

In [6]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [7]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/walk.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

In [8]:
print(args)

{'num_envs': 256, 'num_eval_envs': 32, 'lr': 0.0003, 'max_it': 1000, 'max_grad_norm': 0.4, 'seed': 0, 'system_config': 'humanoid', 'demo_replay_mode': 'threshold', 'threshold': 0.4, 'normalize_observations': True, 'cycle_len': 195, 'ep_len': 39, 'ep_len_eval': 54, 'use_lr_scheduler': True, 'reward_scaling': 0.02, 'rot_weight': 0.5, 'vel_weight': 0.01, 'ang_weight': 0.01, 'deep_mimic_reward_weights': {'w_p': 0.3, 'w_v': 0.1, 'w_e': 0.5, 'w_c': 0.1}, 'deep_mimic_weights_factors': {'w_pose': 2, 'w_angular': 0.005, 'w_efector': 5, 'w_com': 100}, 'ref': 'motions/humanoid3d_walk_duplicated.txt'}


In [9]:
args.deep_mimic_reward_weights.w_p

0.3

In [10]:
from agent_mimic_env.pds_controllers_agents import feedback_pd_controller, stable_pd_controller_action


In [11]:
env_replay,env_eval,env,env_apg=register_mimic_env(args)


#for the env
jit_reset = jax.jit(env.reset)
#jit_step = jax.jit(env.step)
jit_step_custom_env = jax.jit(env.step_custom)
env.set_pd_callback(stable_pd_controller_action)


# #for the eval
jit_reset_eval = jax.jit(env_eval.reset)
jit_step_custom_eval = jax.jit(env_eval.step_custom)
env_eval.set_pd_callback(stable_pd_controller_action)

#for the replay
jit_reset_replay = jax.jit(env_replay.reset)
jit_step_replay= jax.jit(env_replay.step)

#for apg
# jit_reset_apg = jax.jit(env_apg.reset)
# jit_step_custom_apg= jax.jit(env_apg.step_custom)
# env_apg.set_pd_callback(stable_pd_controller_action)





qpos init (195, 35)
qvel init (195, 34)


In [12]:
print(env_eval.sys.opt.timestep)
print(env_eval.dt)

0.002
0.016


In [13]:
print(env_eval.rollout_lenght)
print(env_eval.cycle_len)
print(env_eval.rot_weight)
print(env_eval.vel_weight)
print(env_eval.ang_weight)
print(env_eval.reward_scaling)
print(env_eval.dt)
# print(env_eval.sys.dt)

print(env.rollout_lenght)
print(env.cycle_len)
print(env.rot_weight)
print(env.vel_weight)
print(env.ang_weight)
print(env.reward_scaling)
print(env.dt)
print(env.dt)
print(env.err_threshold)

39
195
0.5
0.01
0.01
0.02
0.016
39
195
0.5
0.01
0.01
0.02
0.016
0.016
0.4


In [27]:
# initialize the state

#for the replay
state = jit_reset_replay(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]

#for the eval
# state = jit_reset_eval(jax.random.PRNGKey(0))
# rollout = [state.pipeline_state]
#for env
# state = jit_reset(jax.random.PRNGKey(0))
# rollout = [state.pipeline_state]

#for apg test
# state = jit_reset_apg(jax.random.PRNGKey(0))
# rollout = [state.pipeline_state]

# grab a 500 steps
n_steps = 500

phi0.046153846153846156


In [15]:
print(state.metrics)
print(state.info)

{'fall': Array(0., dtype=float64), 'pose_error': Array(0., dtype=float64), 'reference_angular': Array(0., dtype=float64, weak_type=True), 'reference_position': Array(0., dtype=float64, weak_type=True), 'reference_rotation': Array(0., dtype=float64, weak_type=True), 'reference_velocity': Array(0., dtype=float64, weak_type=True), 'step_index': Array(9, dtype=int64)}
{'reward_tuple': {'reference_angular': Array(0., dtype=float64, weak_type=True), 'reference_position': Array(0., dtype=float64, weak_type=True), 'reference_rotation': Array(0., dtype=float64, weak_type=True), 'reference_velocity': Array(0., dtype=float64, weak_type=True)}, 'rng': Array([0, 0], dtype=uint32), 'steps': Array(0., dtype=float64, weak_type=True)}


In [26]:
id = 9

id =(195 % 195) /195
id
# cy = (66%65)
# cy

0.0

In [28]:
for i in range(env_replay.rollout_lenght-1):
    
    #print(i)
    #ctrl = -0.1 * jp.ones(env_eval.sys.nu)
    #ctrl = -0.1 * jp.ones(env.sys.nu)
    ctrl = -0.1 * jp.ones(env_replay.sys.nu)
    #ctrl = -0.1 * jp.ones(env_apg.sys.nu)
    #time
    #time = state.pipeline_state.time
    
    #print('time: ',state.pipeline_state.time)    
         
    state = jit_step_replay(state, ctrl)
    #state = jit_step_custom_eval(state, ctrl)
    #state = jit_step(state, ctrl)
    #state = jit_step_custom_env(state,ctrl)
    #state = jit_step_custom_apg(state,ctrl)
    
    #print(state.pipeline_state.geom_xpos)
    #if state.done:
     #   print(time)
      #  break
    #print("Is done", state.done)
    # print("Is rewards", state.reward)
    print('is step', state.metrics['step_index'])
    #print("reward tuple", state.info['reward_tuple'])
    # print("Is ", state.metrics['pose_error'])
    
    rollout.append(state.pipeline_state)

phi0.05128205128205128
is step 10
phi0.05641025641025641
is step 11
phi0.06153846153846154
is step 12
phi0.06666666666666667
is step 13
phi0.07179487179487179
is step 14
phi0.07692307692307693
is step 15
phi0.08205128205128205
is step 16
phi0.08717948717948718
is step 17
phi0.09230769230769231
is step 18
phi0.09743589743589744
is step 19
phi0.10256410256410256
is step 20
phi0.1076923076923077
is step 21
phi0.11282051282051282
is step 22
phi0.11794871794871795
is step 23
phi0.12307692307692308
is step 24
phi0.1282051282051282
is step 25
phi0.13333333333333333
is step 26
phi0.13846153846153847
is step 27
phi0.14358974358974358
is step 28
phi0.14871794871794872
is step 29
phi0.15384615384615385
is step 30
phi0.15897435897435896
is step 31
phi0.1641025641025641
is step 32
phi0.16923076923076924
is step 33
phi0.17435897435897435
is step 34
phi0.1794871794871795
is step 35
phi0.18461538461538463
is step 36
phi0.18974358974358974
is step 37
phi0.19487179487179487
is step 38
phi0.2
is step 39


In [18]:
from some_math.quaternion_diff import *

In [19]:
#6 right writs 9 left writs 12 right ankle, 15 left ankle

print(state.pipeline_state.geom_xpos)


#index 16

[[ 0.          0.          0.        ]
 [ 0.2814414  -0.0135399   0.94205101]
 [ 0.29937734 -0.00502623  1.22739954]
 [ 0.31645127 -0.00592138  1.50576898]
 [ 0.2711695  -0.21985116  1.21612742]
 [ 0.30791153 -0.26587147  0.97227938]
 [ 0.3521438  -0.2824185   0.84160432]
 [ 0.28066436  0.20851551  1.21588093]
 [ 0.29086567  0.2573155   0.96856244]
 [ 0.31671669  0.2806352   0.83404781]
 [ 0.29999011 -0.08805723  0.66862669]
 [ 0.28864182 -0.05446972  0.2628681 ]
 [ 0.29751747 -0.03559797  0.03551529]
 [ 0.35457313  0.08958659  0.67374239]
 [ 0.26744232  0.11211614  0.36537805]
 [ 0.09248359  0.11329693  0.19657547]]


In [20]:
HTML(html.render(env_replay.sys.tree_replace({'opt.timestep': env_replay.dt}), rollout))
#HTML(html.render(env_eval.sys.tree_replace({'opt.timestep': env_eval.dt}), rollout))
#HTML(html.render(env.sys.tree_replace({'opt.timestep': env.dt}), rollout))
#HTML(html.render(env_apg.sys.tree_replace({'opt.timestep': env_apg.dt}), rollout))
